# ReDimNetNoMel Disable bad layers

In [1]:
import torch
print(torch.__version__)
from torchsummary import summary


import torchaudio
import torchaudio.transforms as T

import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from math import floor



2.6.0+cu124


In [2]:
model_name='B0'
train_type='ft_lm'
dataset='vox2'

torch.hub.set_dir('/data/deep/redimnet/models')

original_model = torch.hub.load('IDRnD/ReDimNet', 'ReDimNet', 
                       model_name=model_name, 
                       train_type=train_type, 
                       dataset=dataset)

Using cache found in /data/deep/redimnet/models/IDRnD_ReDimNet_master


/data/deep/redimnet/models/IDRnD_ReDimNet_master
load_res : <All keys matched successfully>


*  ReDimNetWrap expects raw 16 kHz mono audio, exactly 32 000 samples

In [3]:
from torchinfo import summary
summary(original_model, input_size=(1, 32000))

/data/deep/redimnet/models/IDRnD_ReDimNet_master/redimnet/layers/features.py:137: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetWrap                                                 [1, 192]                  --
├─MelBanks: 1-1                                              [1, 60, 134]              --
│    └─Sequential: 2-1                                       [1, 60, 134]              --
│    │    └─Identity: 3-1                                    [1, 32000]                --
│    │    └─PreEmphasis: 3-2                                 [1, 32000]                --
│    │    └─MelSpectrogram: 3-3                              [1, 60, 134]              --
├─ReDimNet: 1-2                                              [1, 600, 134]             --
│    └─Sequential: 2-2                                       [1, 600, 134]             --
│    │    └─Conv2d: 3-4                                      [1, 10, 60, 134]          100
│    │    └─LayerNorm: 3-5                                   [1, 10, 60, 134]          20
│   

we can see MelSpectrogram inside the model ; lets take it outside the model;


In [4]:
for name, module in original_model.named_children():
    print(name, "=>", module)

backbone => ReDimNet(
  (stem): Sequential(
    (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
    (2): to1d()
  )
  (stage0): Sequential(
    (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
    (1): to2d(f=60,c=10)
    (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
    (3): ConvBlock2d(
      (conv_block): ResBasicBlock(
        (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
        (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
        (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      

# create new model

In [5]:
class Conv1dAs2d(nn.Module):
    """
    Replace a Conv1d with an equivalent Conv2d (H = kernel, W = 1)
    so that ONNX shows only Conv2d, which RKNN supports.
    """
    def __init__(self, conv1d: nn.Conv1d):
        super().__init__()

        k, d, s, g = conv1d.kernel_size[0], conv1d.dilation[0], conv1d.stride[0], conv1d.groups

        # --- numeric padding ---
        if isinstance(conv1d.padding, str):        # "same" or "valid"
            if conv1d.padding == "same":
                pad_num = floor(d * (k - 1) / 2)
            else:                                  # "valid"
                pad_num = 0
        else:                                      # already a tuple/int
            pad_num = conv1d.padding[0]

        # Build the Conv2d with weights copied
        self.conv2d = nn.Conv2d(
            in_channels  = conv1d.in_channels,
            out_channels = conv1d.out_channels,
            kernel_size  = (k, 1),
            stride       = (s, 1),
            padding      = (pad_num, 0),
            dilation     = (d, 1),
            groups       = g,
            bias         = conv1d.bias is not None
        )

        with torch.no_grad():
            # (out, in/groups, k) → (out, in/groups, k, 1)
            self.conv2d.weight.copy_(conv1d.weight.unsqueeze(-1))
            if conv1d.bias is not None:
                self.conv2d.bias.copy_(conv1d.bias)

    def forward(self, x):           # x: [B, C, T]
        return self.conv2d(x.unsqueeze(-1)).squeeze(-1)


In [6]:
########################################
# 2) Define a Model Class without MelBanks
########################################
import torch
import torch.nn as nn

class ReDimNetNoMel(nn.Module):
    """
    A wrapper around the original ReDimNetWrap that:
      - Excludes the 'spec' (MelBanks) module
      - Uses 'backbone', 'pool', 'bn', and 'linear'
    We expect a precomputed mel spectrogram as input with shape [B, 1, n_mels, time_frames].
    """
    def __init__(self, original_wrap):
        super().__init__()
        
        # Grab references to the submodules we want to keep
        self.backbone = original_wrap.backbone
        
        # fix problem01
        # list of (stage, block) indices you already know are problematic
        TARGETS = [(0, 6), (1, 8), (2, 8), (3, 9), (4, 7)]
        for s_idx, b_idx in TARGETS:
            for tcm_idx in range(4):
                block = self.backbone.__getattr__(f"stage{s_idx}")[b_idx].tcm[tcm_idx]

                block.dwconvs[0] = Conv1dAs2d(block.dwconvs[0])
                block.pwconv1    = Conv1dAs2d(block.pwconv1)   # 1×1 conv

        
        # Replace ASTP with RKNN-safe version:
        self.pool = original_wrap.pool
        self.bn = original_wrap.bn
        self.linear = original_wrap.linear

    def forward(self, x):
        # x: shape [B, 1, n_mels, time_frames]
        # (1) Pass through the backbone
        x = self.backbone(x)    # shape might become [B, channels, frames] or similar
        print("Backbone output shape:", x.shape)  # ADD THIS LINE
        # (2) Pooling
        x = self.pool(x)        # ASTP => shape likely [B, embedding_dim]
        # (3) BatchNorm
        x = self.bn(x)
        # (4) Final linear => 192-dim (if that's your embedding size)
        x = self.linear(x)
        return x


# Create an instance of our new model that skips the MelBanks front-end
model_no_mel = ReDimNetNoMel(original_model)



In [7]:
model_no_mel.eval()  # <- this line is critical!
dummy = torch.randn(1, 1, 60, 200)
model_no_mel(dummy)

Backbone output shape: torch.Size([1, 600, 200])


tensor([[ 0.7321,  0.8093, -3.4446, -3.0097, -2.2613, -0.6797,  0.7720,  1.3006,
          1.9086,  2.3522,  1.0694, -0.4370, -3.8690,  0.3531, -3.7271,  0.7892,
         -3.0142,  0.2123,  2.8541, -2.6768, -0.5221,  1.8836, -1.0156,  1.3764,
         -0.1948, -1.4629,  0.9342, -1.1070,  0.3005,  1.8699,  0.9252, -4.9514,
          0.0347,  1.3432, -1.5038,  1.2418,  1.5106, -2.8904, -1.6833, -4.0154,
         -2.9377,  2.1710,  3.7802, -4.3935,  3.6596,  1.0019, -1.6085,  0.7020,
         -1.3796,  0.0691,  0.2980, -0.9571, -3.4507,  2.4188, -4.7347, -1.2065,
         -1.3179, -4.4689,  1.0363, -2.3271, -1.8628, -2.7428, -0.6891, -0.9601,
          0.7414,  1.8051, -3.0718,  4.2250, -0.4580, -0.2536,  1.4904, -0.0480,
         -4.3755, -1.9402,  0.7589, -5.1639, -2.7502, -0.6591, -4.5342,  0.0175,
         -3.1154,  6.1118,  1.3613, -3.4238,  2.0232,  0.3731,  4.0782,  0.7025,
          0.2207,  1.6413, -0.0135,  1.3388,  3.2963,  3.8507,  0.5965,  1.0606,
          1.6012,  2.8436,  

## layres debug

problematic layer tree is:

```
TimeContextBlock1d
├── red_dim_conv (Sequential)
│   ├── Conv1d(600 → 60, kernel_size=1)
│   └── LayerNorm(C=60, data_format=channels_first)
├── tcm (Sequential)
│   ├── ConvNeXtLikeBlock (kernel=7)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=7, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=19)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=19, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=31)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=31, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   ├── ConvNeXtLikeBlock (kernel=59)
│   │   ├── dwconvs: Conv1d(60 → 60, kernel_size=59, groups=60)
│   │   ├── norm: BatchNorm1d(60)
│   │   ├── act: GELU
│   │   └── pwconv1: Conv1d(60 → 60, kernel_size=1)
│   └── TransformerEncoderLayer
│       ├── attention (MultiHeadAttention)
│       │   ├── k_proj: Linear(60 → 60)
│       │   ├── v_proj: Linear(60 → 60)
│       │   ├── q_proj: Linear(60 → 60)
│       │   └── out_proj: Linear(60 → 60)
│       ├── layer_norm: LayerNorm(60)
│       ├── feed_forward
│       │   ├── intermediate_dropout: Dropout(0.0)
│       │   ├── intermediate_dense: Linear(60 → 60)
│       │   ├── intermediate_act_fn: NewGELUActivation
│       │   ├── output_dense: Linear(60 → 60)
│       │   └── output_dropout: Dropout(0.0)
│       └── final_layer_norm: LayerNorm(60)
└── exp_dim_conv: Conv1d(60 → 600, kernel_size=1)
```

In [8]:
for name, module in model_no_mel.named_modules():
    if isinstance(module, nn.LayerNorm):
        print("❌ Still has LayerNorm at:", name)

❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage0.6.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage1.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage2.8.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage3.9.tcm.4.final_layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.layer_norm
❌ Still has LayerNorm at: backbone.stage4.7.tcm.4.final_layer_norm


In [9]:
print("stage0.6 =", model_no_mel.backbone.stage4[7].tcm[0]) 

stage0.6 = ConvNeXtLikeBlock(
  (dwconvs): ModuleList(
    (0): Conv1dAs2d(
      (conv2d): Conv2d(60, 60, kernel_size=(7, 1), stride=(1, 1), padding=(3, 0), groups=60)
    )
  )
  (norm): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): GELU(approximate='none')
  (pwconv1): Conv1dAs2d(
    (conv2d): Conv2d(60, 60, kernel_size=(1, 1), stride=(1, 1))
  )
)


## FP16 check


In [10]:
import copy

with torch.no_grad():
    fp16_net = copy.deepcopy(model_no_mel).half().eval()
    ok = torch.isfinite(fp16_net(dummy.half())).all()
    print('safe in pure FP16?', ok)

Backbone output shape: torch.Size([1, 600, 200])
safe in pure FP16? tensor(True)


## info

In [11]:
model_no_mel.eval()


ReDimNetNoMel(
  (backbone): ReDimNet(
    (stem): Sequential(
      (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): LayerNorm(C=(10,), data_format=channels_first, eps=1e-06)
      (2): to1d()
    )
    (stage0): Sequential(
      (0): weigth1d(w=(1, 1, 1, 1),sequential=False)
      (1): to2d(f=60,c=10)
      (2): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
      (3): ConvBlock2d(
        (conv_block): ResBasicBlock(
          (conv1): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv1pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=10, bias=False)
          (conv2pw): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1))
          (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_st

In [12]:
summary(model_no_mel, (1, 1, 60, 200))


Backbone output shape: torch.Size([1, 600, 200])


Layer (type:depth-idx)                                       Output Shape              Param #
ReDimNetNoMel                                                [1, 192]                  --
├─ReDimNet: 1-1                                              [1, 600, 200]             --
│    └─Sequential: 2-1                                       [1, 600, 200]             --
│    │    └─Conv2d: 3-1                                      [1, 10, 60, 200]          100
│    │    └─LayerNorm: 3-2                                   [1, 10, 60, 200]          20
│    │    └─to1d: 3-3                                        [1, 600, 200]             --
│    └─Sequential: 2-2                                       [1, 600, 200]             --
│    │    └─weigth1d: 3-4                                    [1, 600, 200]             (1)
│    │    └─to2d: 3-5                                        [1, 10, 60, 200]          --
│    │    └─Conv2d: 3-6                                      [1, 10, 60, 200]          110
│ 

## Utility Function for WAV -> MelSpectrogram

In [13]:
def pad_or_crop_logmel(log_mel, target_frames=200):
    """
    Ensures log_mel is shaped [1, n_mels, target_frames] by:
    - Padding with zeros on the right if too short
    - Center-cropping if too long
    """
    B, M, T = log_mel.shape
    if T < target_frames:
        pad_amt = target_frames - T
        log_mel = F.pad(log_mel, (0, pad_amt))  # pad at end
        print(f"Padding log_mel from {T} to {target_frames} frames")
    elif T > target_frames:
        start = (T - target_frames) // 2
        log_mel = log_mel[:, :, start:start + target_frames]
        print(f"Cropping log_mel from {T} to {target_frames} frames")
    return log_mel


def waveform_to_logmel(
    waveform: torch.Tensor,
    sample_rate: int = 16000,
    n_fft: int = 512,
    hop_length: int = 160,
    n_mels: int = 60,       ## 72 for vox2 ;  60 for B0
    f_min: float = 20.0,
    f_max: float = 8000.0,
    preemphasis_alpha: float = 0.97,
    target_frames=200
):
    """
    Reproduces the main logic of 'NormalizeAudio', 'PreEmphasis',
    and 'MelSpectrogram' from the 'MelBanks' layer.
    """

    # 1) NormalizeAudio
    waveform = waveform / (waveform.abs().max() + 1e-8)

    # 2) PreEmphasis
    shifted = torch.roll(waveform, shifts=1, dims=1)
    waveform_preemph = waveform - preemphasis_alpha * shifted
    # fix first sample
    waveform_preemph[:, 0] = waveform[:, 0]

    # 3) MelSpectrogram
    mel_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=sample_rate,
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        f_min=f_min,
        f_max=f_max,
        power=2.0,
        center=False
    )
    mel_spec = mel_transform(waveform_preemph)  # shape: [channel=1, n_mels, time_frames]

    # Log scale
    # log_mel = torch.log(mel_spec + 1e-6)
    log_mel = np.log(np.clip(mel_spec, a_min=1e-2, a_max=None))
    
    # 5) Pad/crop to fixed number of frames
    log_mel = pad_or_crop_logmel(log_mel, target_frames=target_frames)
    
    print("Log-mel shape:", log_mel.shape)  # Expect [1, 1, 60, 200]
    return log_mel  # shape: [1, n_mels, time_frames]

In [14]:
def example_inference(wav_path: str):
    # (a) Load audio
    waveform, sr = torchaudio.load(wav_path)  # shape: [channels, time]
    # If stereo, select one channel, or average:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # (b) Convert to log-mel
    log_mel = waveform_to_logmel(waveform, sample_rate=sr)
    # Now log_mel shape is [1, n_mels, time_frames].

    # (c) Model expects a batch, so add batch dim => [B=1, 1, n_mels, time_frames]
    log_mel = log_mel.unsqueeze(0)

    # (d) Forward pass
    with torch.no_grad():
        embedding = model_no_mel(log_mel)  # shape typically [1, 192] or so

    print("Embedding shape:", embedding.shape)
    #print("Embedding:", embedding)
    return embedding

In [15]:
# Compute similarity between two embeddings
def cosine_similarity(embedding1, embedding2):
    return F.cosine_similarity(embedding1, embedding2).item()

def cosine_similarity_numpys(emb1: np.ndarray, emb2: np.ndarray) -> float:
    """
    Compute cosine similarity between two vectors of shape (D,) or (1, D).
    """
    # If shape is (1, D), flatten to (D,)
    v1 = emb1.flatten()
    v2 = emb2.flatten()

    # dot product
    dot = np.dot(v1, v2)
    # norms
    norm1 = np.linalg.norm(v1)
    norm2 = np.linalg.norm(v2)

    # Add a small epsilon in case of very small norms
    sim = dot / (norm1 * norm2 + 1e-8)
    return sim


In [16]:
embed1 = example_inference("testRob1.wav")
embed2 = example_inference("testRob2.wav")
embed3 = example_inference("testme1.wav")
embed4 = example_inference("testme2.wav")


Cropping log_mel from 219 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])


/tmp/ipykernel_227229/1275554942.py:59: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  log_mel = np.log(np.clip(mel_spec, a_min=1e-2, a_max=None))


Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])
Log-mel shape: torch.Size([1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])


/data/proj/voice/pyvoice_venv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (60) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


Cropping log_mel from 525 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])
Cropping log_mel from 993 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])
Backbone output shape: torch.Size([1, 600, 200])
Embedding shape: torch.Size([1, 192])


In [17]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to me   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")

Similarity (robot to robot): 0.8067655563354492
Similarity (robot to me   ): 0.19459125399589539
Similarity (me 1 to me 2  ): 0.719450056552887


/tmp/ipykernel_227229/3060022475.py:14: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  dot = np.dot(v1, v2)


## store

In [18]:
import onnx

def export_to_onnx(model, onnx_path="ReDimNet_no_mel.onnx"):
    model.eval()
    
    # Create a dummy input with shape matching [B=1, 1, n_mels=72, time_frames=200] (example)
    dummy_input = torch.randn(1, 1, 60, 200)
    
    #  fixed-length segments 
    torch.onnx.export(
        model_no_mel,
        dummy_input,
        onnx_path,
        input_names=["log_mel"],
        output_names=["embedding"],
        opset_version=11
    )
    print("Exported to", onnx_path)
    



In [19]:
export_to_onnx(model_no_mel,onnx_path = "ReDimNet_no_mel.onnx")
!ls -lah ReDimNet_no_mel.onnx

Backbone output shape: torch.Size([1, 600, 200])
Exported to ReDimNet_no_mel.onnx
-rw-rw-r-- 1 vlad vlad 4.2M Jun 15 17:53 ReDimNet_no_mel.onnx


In [20]:
def scale_last_linear(model: nn.Module, scale: float = 1/32):
    """
    Locate the Linear(1200→192) layer in ReDimNetNoMel and scale its
    weights & bias by `scale`.  Returns the layer so you can check.
    """
    target = None
    for name, mod in model.named_modules():
        if isinstance(mod, nn.Linear) and mod.in_features == 1200 and mod.out_features == 192:
            target = mod
            print(f"✔ found head: {name}  (will scale by {scale})")
            break
    if target is None:
        raise RuntimeError("Linear(1200→192) not found!")

    with torch.no_grad():
        target.weight.mul_(scale)
        target.bias.mul_(scale)
    return target

In [21]:
scale = 1/32
head = scale_last_linear(model_no_mel, scale)

export_to_onnx(model_no_mel,onnx_path = "ReDimNet_no_mel_scaled.onnx")
!ls -lah ReDimNet_no_mel_scaled.onnx

✔ found head: linear  (will scale by 0.03125)
Backbone output shape: torch.Size([1, 600, 200])
Exported to ReDimNet_no_mel_scaled.onnx
-rw-rw-r-- 1 vlad vlad 4.2M Jun 15 17:53 ReDimNet_no_mel_scaled.onnx


## verify

In [22]:
import onnx
onnx_model = onnx.load("ReDimNet_no_mel.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model is valid!")


ONNX model is valid!


In [23]:
print(type(model_no_mel))
print(isinstance(model_no_mel, nn.Module))

<class '__main__.ReDimNetNoMel'>
True


In [24]:
type(onnx_model)

onnx.onnx_ml_pb2.ModelProto

In [25]:
import onnxruntime as ort

def run_inference_onnx(onnx_path, wav_path):
    """
    Loads an audio file, converts to log-mel, and runs inference
    in an ONNX session. Returns the embedding as a NumPy array.
    """
    #######################################
    # 1) Load your ONNX model
    #######################################
    # (Optional) onnx.checker to confirm it’s valid
    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    print(f"Loaded and checked ONNX model from: {onnx_path}")

    # Create an inference session
    session = ort.InferenceSession(onnx_path)

    # Usually we retrieve the first input & output name
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    #######################################
    # 2) Load audio, get log-mel
    #######################################
    waveform, sr = torchaudio.load(wav_path)
    # If multi-channel, downmix:
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    log_mel = waveform_to_logmel(waveform, sample_rate=sr)
    # Insert a batch dimension => shape [B, 1, n_mels, frames]
    log_mel = log_mel.unsqueeze(0)  # => [1, 1, n_mels, time_frames]

    #######################################
    # 3) ONNX Inference
    #######################################
    # Convert to NumPy for ONNX runtime
    log_mel_np = log_mel.cpu().numpy()
    # Run inference
    outputs = session.run([output_name], {input_name: log_mel_np})
    # outputs is a list; typically we want the first item
    embedding = outputs[0]  # shape is [1, embedding_dim]

    print("Embedding shape:", embedding.shape)
    # print("Embedding data:\n", embedding)
    return embedding

In [26]:
onnx_model_path = "ReDimNet_no_mel.onnx"

embed1 = run_inference_onnx(onnx_model_path, "testRob1.wav")
embed2 = run_inference_onnx(onnx_model_path, "testRob2.wav")
embed3 = run_inference_onnx(onnx_model_path, "testme1.wav")
embed4 = run_inference_onnx(onnx_model_path, "testme2.wav")


Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Cropping log_mel from 219 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])


/tmp/ipykernel_227229/1275554942.py:59: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  log_mel = np.log(np.clip(mel_spec, a_min=1e-2, a_max=None))


Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Log-mel shape: torch.Size([1, 60, 200])
Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Cropping log_mel from 525 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])
Embedding shape: (1, 192)
Loaded and checked ONNX model from: ReDimNet_no_mel.onnx
Cropping log_mel from 993 to 200 frames
Log-mel shape: torch.Size([1, 60, 200])
Embedding shape: (1, 192)


In [27]:
print(f"Similarity (robot to robot): {cosine_similarity_numpys(embed1, embed2)}")
print(f"Similarity (robot to me   ): {cosine_similarity_numpys(embed1, embed3)}")
print(f"Similarity (me 1 to me 2  ): {cosine_similarity_numpys(embed3, embed4)}")

Similarity (robot to robot): 0.8067656755447388
Similarity (robot to me   ): 0.1945914775133133
Similarity (me 1 to me 2  ): 0.7194505333900452


## cal fake data

* run in rknn docker:

``` bash

import os
import numpy as np
import torch

# Directory for calibration inputs
os.makedirs("calib_npy", exist_ok=True)

# Create 100 dummy log-mel tensors
for i in range(10):
    log_mel = torch.randn(1, 1, 60, 200).numpy().astype(np.float32)
    np.save(f"calib_npy/sample_{i}.npy", log_mel)

# Write dataset.txt listing all paths
with open("dataset.txt", "w") as f:
    for i in range(10):
        f.write(f"calib_npy/sample_{i}.npy\n")


````

## converts

* python convert.py ReDimNet_no_mel.onnx  rk3588 fp ReDimNet_no_mel.rknn 
* python convert.py ReDimNet_no_mel.onnx  rv1106 i8  ReDimNet_no_mel.rknn